In [ ]:
# Import necessary libraries

from operator import index

import pandas as pd

!pip install langdetect
import langdetect as detect

import numpy as np

import os

import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

from google.colab import drive

# Download necessary data for natural language processing tasks

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('omw-1.4')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=b6a7efec00c3780867b271fb01b9d9bece2ed42292ebedcaee7fb34426b56e6c
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Read in spam data from CSV file
df = pd.read_csv('/content/drive/MyDrive/spam.csv',
                 sep=',', header=0, on_bad_lines='skip', encoding = "ISO-8859-1")

In [ ]:
df.head()

,v1,v2,TokenSMS,StopTokenSMS,LengthTokenSMS,LemTokenSMS
0,0,go until jurong point crazy available only in ...,"[go, until, jurong, point, crazy, available, o...","[go, jurong, point, crazy, available, bugis, n...",jurong point crazy available bugis great world...,jurong point crazy available bugis great world...
1,0,ok lar joking wif u oni,"[ok, lar, joking, wif, u, oni]","[ok, lar, joking, wif, u, oni]",lar joking wif oni,lar joking wif oni
2,1,free entry in a wkly comp to win fa cup final ...,"[free, entry, in, a, wkly, comp, to, win, fa, ...","[free, entry, wkly, comp, win, fa, cup, final,...",free entry wkly comp win cup final tkts may te...,free entry wkly comp win cup final tkts may te...
3,0,u dun say so early hor u c already then say,"[u, dun, say, so, early, hor, u, c, already, t...","[u, dun, say, early, hor, u, c, already, say]",dun say early hor already say,dun say early hor already say
4,0,nah i don't think he goes to usf he lives arou...,"[nah, i, do, n't, think, he, goes, to, usf, he...","[nah, n't, think, goes, usf, lives, around, th...",nah n't think goes usf lives around though,nah n't think goes usf lives around though


In [ ]:
# Drop any "Unnamed" columns

unnamed_cols = df.columns[df.columns.str.contains("Unnamed")]
df.drop(columns=unnamed_cols, inplace=True)

In [ ]:
# Define regular expression variables to remove from SMS messages

reg_vars = ['http\S+', 'www\S+', 'https\S+', '\W\s+', '\d+', '\t+', '\d+', '\-+', '\\+', '\/+', '\"+', '\#+', '\++', '\@+', '\$+', '\%+', '\^+', '\&+', '\*+', '\(+', '\)+', '\[+', '\]+', '\{+', '\}+', '\|+', '\;+', '\:+', '\<+', '\>+', '\?+', '\,+', '\.+', '\=+', '\_+', '\~+', '\`+', '\s+']

df.replace(reg_vars, ' ', regex=True, inplace=True)

df.drop_duplicates(inplace=True)

df.replace('', np.nan, inplace=True)

df.dropna(inplace=True)

In [ ]:
# Remove rows with non-ASCII characters from the dataframe

df = df[df['v2'].map(lambda x: x.isascii())]

In [ ]:
# Drop non-English rows from the dataframe

for i in range(len(df)):
    try:
        ['v2'][i] = detect.detect(df['v2'][i])
        if df['v2'][i] != 'en':
            df.drop(i, inplace=True, index=False)
    except:
        pass

In [ ]:
# Convert all the text data into lowercase

df['v2'] = df['v2'].astype(str).str.lower()

In [ ]:
# Retrieve a list of English stop words and assign it to a var

stopwords = nltk.corpus.stopwords.words("english")

In [ ]:
# Tokenize the SMS messages in the dataframe

df['TokenSMS'] = df.apply(lambda column: nltk.word_tokenize(column['v2']), axis=1)

In [ ]:
df['TokenSMS'].head(2)

0    [go, until, jurong, point, crazy, available, o...
1                       [ok, lar, joking, wif, u, oni]
Name: TokenSMS, dtype: object

In [ ]:
# Create a column containing the Tokenized words without the stopwords

df['StopTokenSMS'] = df['TokenSMS'].apply(lambda x: [item for item in x if item not in stopwords])

In [ ]:
# Create a coolumn containting the StopTokenSMS text with words less than 2 characters

df['LengthTokenSMS'] = df['StopTokenSMS'].apply(lambda x: ' '.join([item for item in x if len(item)>2]))

In [ ]:
# Instantiate and the assigne a variable to the WordNetLemmatizer class

wordnet_lem = WordNetLemmatizer()

In [ ]:
# Create a new column which contains the lemmatized words

df['LemTokenSMS'] = df['LengthTokenSMS'].apply(wordnet_lem.lemmatize)

In [ ]:
# Clean all dataframes again

reg_vars = ['http\S+', 'www\S+', 'https\S+', '\W\s+', '\d+', '\t+', '\d+', '\-+', '\\+', '\/+', '\"+', '\#+', '\++', '\@+', '\$+', '\%+', '\^+', '\&+', '\*+', '\(+', '\)+', '\[+', '\]+', '\{+', '\}+', '\|+', '\;+', '\:+', '\<+', '\>+', '\?+', '\,+', '\.+', '\=+', '\_+', '\~+', '\`+', '\s+']

df.replace(reg_vars, ' ', regex=True, inplace=True)

df.replace('', np.nan, inplace=True)

df.dropna(inplace=True)

In [ ]:
# Initialize a CountVectorizer object

cv = CountVectorizer()

# fit_transform the data to a numpy array

x = cv.fit_transform(df['LemTokenSMS']).toarray()

In [ ]:
x.shape

(4672, 6869)

In [ ]:
# Replace 'spam' with 1 and 'ham' with 0
df['v1'] = df['v1'].replace({'spam': 1, 'ham': 0})

In [ ]:
# Store the labels in  y

y = df['v1'].values

y.shape

# convert y to to int type

y = y.astype('int')

In [ ]:
# Split the data into a training set and testing set

x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
# Initialize a MultinomialNB objec

mnb=MultinomialNB()

# Training the classifier and making predictions on the test data

mnb.fit(x_train,y_train)
y_pred=mnb.predict(x_test)
print(accuracy_score(y_test,y_pred))

0.9689839572192513


In [ ]:
import pickle

# Save the model to a file
with open('model.pkl', 'wb') as file:
    pickle.dump(mnb, file)

# Save the model to a file
with open('cv.pkl', 'wb') as file:
    pickle.dump(cv, file)

In [ ]:
from google.colab import files

files.download('model.pkl')
files.download('cv.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
feature_log_prob = mnb.feature_log_prob_
class_log_prior = mnb.class_log_prior_

In [ ]:
weights = feature_log_prob.T
intercepts = class_log_prior

In [ ]:
input_dim = x_train.shape[1]  # Dimension of input features
output_dim = len(set(y_train))

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(input_dim,)),
    tf.keras.layers.Dense(output_dim, use_bias=True, activation='softmax')
])

In [ ]:
model.layers[0].set_weights([weights, intercepts])

In [ ]:
# Compile the TensorFlow model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
tf.saved_model.save(model, 'tf_model')

In [ ]:
# Convert the TensorFlow model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_saved_model('tf_model')
tflite_model = converter.convert()

In [ ]:
with open('model.tflite', 'wb') as file:
    file.write(tflite_model)

In [ ]:
# Create a preprocessing function to process new text

def clean_dataframe(df):
    reg_vars = ['http\S+', 'www\S+', 'https\S+', '\W\s+', '\d+', '\t+', '\d+', '\-+', '\\+', '\/+', '\"+', '\#+', '\++', '\@+', '\$+', '\%+', '\^+', '\&+', '\*+', '\(+', '\)+', '\[+', '\]+', '\{+', '\}+', '\|+', '\;+', '\:+', '\<+', '\>+', '\?+', '\,+', '\.+', '\=+', '\_+', '\~+', '\`+', '\s+']
    df['text'].replace(reg_vars, ' ', regex=True, inplace=True)
    df['text'] = df['text'].astype(str).str.lower()
    df['text'] = df.apply(lambda column: nltk.word_tokenize(column['text']), axis=1)
    stopwords = nltk.corpus.stopwords.words('english')
    df['text'] = df['text'].apply(lambda x: [item for item in x if item not in stopwords])
    df['text'] = df['text'].apply(lambda x: ' '.join([item for item in x if len(item)>2]))
    df['text'] = df['text'].apply(wordnet_lem.lemmatize)


In [ ]:
data = [{"text": "Urgent dont miss news dun say so early hor... U c already then say lucky man"}]

ndf = pd.DataFrame(data)

clean_dataframe(ndf)

In [ ]:
newtext = cv.transform(ndf['text']).toarray()
prediction = mnb.predict(newtext)
print(prediction)

[0]


In [ ]:
import numpy as np
import tensorflow as tf
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
import re

def clean_text(text):
    # Define regex patterns for cleaning
    reg_vars = ['http\S+', 'www\S+', 'https\S+', '\W\s+', '\d+', '\t+', '\d+', '\-+', '\\+', '\/+', '\"+', '\#+', '\++', '\@+', '\$+', '\%+', '\^+', '\&+', '\*+', '\(+', '\)+', '\[+', '\]+', '\{+', '\}+', '\|+', '\;+', '\:+', '\<+', '\>+', '\?+', '\,+', '\.+', '\=+', '\_+', '\~+', '\`+', '\s+']
    # Remove regex patterns
    for pattern in reg_vars:
        text = re.sub(pattern, ' ', text)
    # Convert to lowercase
    text = text.lower()
    # Tokenize text
    tokens = nltk.word_tokenize(text)
    # Remove stopwords
    stopwords = set(nltk.corpus.stopwords.words('english'))
    tokens = [word for word in tokens if word not in stopwords]
    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Join tokens back into text
    text = ' '.join(tokens)
    return text


# Load the TensorFlow Lite model
interpreter = tf.lite.Interpreter(model_path='model.tflite')
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Preprocess new text data
data = [{"text": "Urgent dont miss news dun say so early hor... U c already then say lucky man"}]
ndf = pd.DataFrame(data)
ndf['text'] = ndf['text'].apply(clean_text)

# Convert text to TF-IDF vectors
new_text_features = cv.transform(ndf['text']).toarray()

# Perform inference
predictions = []
for x in new_text_features:
    # Convert input data to FLOAT32
    x = x.astype(np.float32)

    # Set input tensor
    interpreter.set_tensor(input_details[0]['index'], np.expand_dims(x, axis=0))

    # Run inference
    interpreter.invoke()

    # Get output tensor
    output = interpreter.get_tensor(output_details[0]['index'])

    # Assuming output is the predicted class probabilities
    predictions.append(output)

# Post-processing if necessary, such as converting probabilities to class labels

# Example of printing predictions
for i, pred in enumerate(predictions):
    print(f"Sample {i+1} prediction: {pred}")


Sample 1 prediction: [[1.000000e+00 9.913665e-09]]


In [ ]:
# Get the predicted output class
predicted_class = np.argmax(predictions[0])  # Assuming there's only one prediction
print("Predicted class:", predicted_class)

Predicted class: 0
